# Install Libraries

In [1]:
!pip install selenium

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


# Imports

In [43]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from time import sleep
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

# Work

In [2]:
binary = r'C:\Program Files\Mozilla Firefox\firefox.exe'
options = Options()
options.headless = True
options.binary = binary

cap = DesiredCapabilities().FIREFOX
cap["marionette"] = True #optional

driver = webdriver.Firefox(options=options, capabilities=cap)
#driver = webdriver.Firefox(capabilities=cap)
driver.get("http://diariooficial.rn.gov.br/dei/dorn3/")

In [3]:
materias = []

In [4]:
data = driver.find_element_by_class_name('ui-datepicker-days-cell-over.ui-datepicker-current-day')
ano = data.get_attribute('data-year')
mes = data.get_attribute('data-month')
dia = data.find_element_by_tag_name('a').get_attribute('text')
data_formatada = (dia + '/0' + mes + '/' + ano if int(mes) < 10 else dia + '/' + mes + '/' + ano)
data_formatada

'28/08/2019'

In [5]:
def processar_linhas(link, text, blank):
    caminho = driver.find_element_by_id("lblSessaoPath").text
    lista_caminho = caminho.split("\\")[:-1]
    if blank != 0:
        [lista_caminho.append('') for b in range(blank)]
    if lista_caminho[0] == "DIÁRIO DA JUSTIÇA":
        lista_caminho.insert(0,'DIÁRIO OFICIAL')
    lista_caminho.insert(0, data_formatada)
    lista_caminho.append(text)
    lista_caminho.append(link)
    print(lista_caminho)
    return lista_caminho

In [6]:
def navegar(link, blank=5):
    link.click()
    global materias
    subsessoes = []
    try:
        subsessoes = driver.find_element_by_id("lblQtdMateriasPublicadas")
        if subsessoes.text == 'Nenhuma matéria encontrada nessa data para essa seção.':
            materia = processar_linhas(link=subsessoes.text, text='Não Possui', blank=blank)
            materias.append(materia)
            return []
        tabela = driver.find_element_by_id("dgDocumentos")
        files = tabela.find_elements_by_tag_name("a")
        for f in files:
            link = f.get_attribute("href")
            materia = processar_linhas(link=link, text=f.text, blank=blank)
            materias.append(materia)
        try:
            tabela = driver.find_element_by_id("dgSubSessoes")
            links = tabela.find_elements_by_tag_name("a")
            return links
        except:
            return []
    except:
        tabela = driver.find_element_by_id("dgSubSessoes")
        links = tabela.find_elements_by_tag_name("a")
    return links

#elements = driver.find_element_by_id("Menu1_dgMenu_ctl04_LinkButton1")
#teste = navegar(elements)
#materias

In [7]:
def crawl_diario(link_sessao):
    #sleep(1)
    sub1 = navegar(link_sessao)
    if len(sub1) != 0:
        id1 = ["dgSubSessoes_ctl0"+str(i)+"_Linkbutton1" if i < 10 
               else "dgSubSessoes_ctl"+str(i)+"_Linkbutton1" for i in range(2,len(sub1)+2)]
        for i1 in id1:
            #sleep(1)
            #print(i1)
            s1 = driver.find_element_by_id(i1)
            sub2 = navegar(s1, blank=4)
            if len(sub2) == 0:
                #sleep(1)
                driver.back()
            else:
                id2 = ["dgSubSessoes_ctl0"+str(i)+"_Linkbutton1" if i < 10 
                       else "dgSubSessoes_ctl"+str(i)+"_Linkbutton1" for i in range(2,len(sub2)+2)]
                for i2 in id2:
                    #sleep(1)
                    s2 = driver.find_element_by_id(i2)
                    sub3 = navegar(s2, blank=3)
                    if len(sub3) == 0:
                        #sleep(1)
                        driver.back()
                    else:
                        id3 = ["dgSubSessoes_ctl0"+str(i)+"_Linkbutton1" if i < 10 
                               else "dgSubSessoes_ctl"+str(i)+"_Linkbutton1" for i in range(2,len(sub3)+2)]
                        for i3 in id3:
                            #sleep(1)
                            s3 = driver.find_element_by_id(i3)
                            sub4 = navegar(s3, blank=2)
                            if len(sub4) == 0:
                                #sleep(1)
                                driver.back()
                            else:
                                id4 = ["dgSubSessoes_ctl0"+str(i)+"_Linkbutton1" if i < 10 
                                       else "dgSubSessoes_ctl"+str(i)+"_Linkbutton1" for i in range(2,len(sub4)+2)]
                                for i4 in id4:
                                    #sleep(1)
                                    s4 = driver.find_element_by_id(i4)
                                    sub5 = navegar(s4, blank=1)
                                    if len(sub5) == 0:
                                        #sleep(1)
                                        driver.back()
                                    else:
                                        id5 = ["dgSubSessoes_ctl0"+str(i)+"_Linkbutton1" if i < 10 
                                               else "dgSubSessoes_ctl"+str(i)+"_Linkbutton1" for i in range(2,len(sub5)+2)]
                                        for i5 in id5:
                                            #sleep(1)
                                            s5 = driver.find_element_by_id(i5)
                                            sub6 = navegar(s5, blank=0)
                                            if len(sub6) == 0:
                                                #sleep(1)
                                                driver.back()
                                            if i5 == id5[-1]:
                                                #sleep(1)
                                                driver.back()
                                    if i4 == id4[-1]:
                                        #sleep(1)
                                        driver.back()
                            if i3 == id3[-1]:
                                #sleep(1)
                                driver.back()
                    if i2 == id2[-1]:
                        #sleep(1)
                        driver.back()
            if i1 == id1[-1]:
                #sleep(1)
                driver.back()

segunda versão

In [8]:
elements = ["Menu1_dgMenu_ctl0" + str(i) + "_LinkButton1" for i in range(2,10)]
#lista = navegar(sessoes[7])
#lista = []
for e in elements:
    s = driver.find_element_by_id(e)
    crawl_diario(s)
print(materias)
#lista

['28/08/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (ASSECOM)', 'DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI)', '', '', 'DEI - EXTRATO DISPENSA DE LICITAÇÃO (TECPEL).doc', 'http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20190928&id_doc=660593']
['28/08/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (ASSECOM)', 'DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI)', 'DEI ASSEJUR', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['28/08/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'CONSELHO DE DESENVOLVIMENTO DO ESTADO (CDE)', '', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['28/08/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'CONSULTORIA GERAL DO ESTADO (CGE)', '', '', '', 'Não Possui', 'Nenhuma matéria encontr

['28/08/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DA INFRA-ESTRUTURA', 'DEPARTAMENTO ESTADUAL DE TRÂNSITO (DETRAN)', '', '', 'Extrato de Contrato', 'http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20190928&id_doc=660540']
['28/08/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DA JUSTIÇA E DA CIDADANIA', 'CONSELHO ESTADUAL DE DIREITOS HUMANOS E CIDADANIA (COEDHUCI)', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['28/08/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DE ADMINISTRAÇÃO E DOS RECURSOS HUMANOS', '', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['28/08/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DE ASSUNTOS FUNDIÁRIOS, COLONIZAÇÃO E APOIO À REFORMA AGRÁRIA', '', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa

['28/08/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DE ESTADO DO PLANEJAMENTO E DAS FINANÇAS (SEPLAN)', 'AGÊNCIA REGULADORA DE SERVIÇOS PÚBLICOS DO RN (ARSEP)', '', '', 'PRIMEIRO TERMO ADITIVO - JMT', 'http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20190928&id_doc=660497']
['28/08/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DE ESTADO DO PLANEJAMENTO E DAS FINANÇAS (SEPLAN)', 'COORDENADORIA DE ESTUDOS SOCIOECONÔMICOS (CES)', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['28/08/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DE ESTADO DO PLANEJAMENTO E DAS FINANÇAS (SEPLAN)', 'EMPRESA DE GESTÃO DE ATIVOS DO RIO GRANDE DO NORTE (EMGERN)', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['28/08/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECR

['28/08/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DO DESENVOLVIMENTO ECONÔMICO', 'COMPANHIA POTIGUAR DE GÁS (POTIGAS)', '', '', 'POTIGÁS - EXTRATOS DE CONTRATOS - PABLO ALVES DE SOUZA XAVIER e OUTROS.doc', 'http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20190928&id_doc=660639']
['28/08/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DO DESENVOLVIMENTO ECONÔMICO', 'FUNCAÇÃO DE APOIO À PESQUISA DO ESTADO DO RN (FAPERN)', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['28/08/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DO DESENVOLVIMENTO ECONÔMICO', 'INSTITUTO DE PESOS E MEDIDAS DO RN (IPEM)', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['28/08/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DO DESENVOLVIMENTO ECONÔMICO', 'JUNTA C

['28/08/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'SECRETARIAS DE ESTADO', 'SECRETARIA DA SEGURANÇA PÚBLICA E DA DEFESA SOCIAL ', 'DIRETORIA DE APOIO LOGISTICO ( DAL )', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['28/08/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'SECRETARIAS DE ESTADO', 'SECRETARIA DA SEGURANÇA PÚBLICA E DA DEFESA SOCIAL ', 'FUNDO ESPECIAL DE SEGURANCA PUBLICA', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['28/08/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'SECRETARIAS DE ESTADO', 'SECRETARIA DA SEGURANÇA PÚBLICA E DA DEFESA SOCIAL ', 'GABINETE DO DELEGADO GERAL(GDG/PC)', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['28/08/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'SECRETARIAS DE ESTADO', 'SECRETARIA DA SEGURANÇA PÚBLICA E DA DEFESA SOCIAL ', 'INSTITUTO TÉCNICO-CIENTÍFICO DA POLÍCIA (ITEP) ', '', '', 'Portaria 219 - 2019 - GDG - ITEP. Regulamento a coleta 

...

['28/08/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'JUIZADOS ESPECIAIS CÍVEIS E CRIMINAIS', '2ª TURMA RECURSAL DOS JUIZADOS ESPECIAIS', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['28/08/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'JUIZADOS ESPECIAIS CÍVEIS E CRIMINAIS', 'ATOS DO JUIZ COORDENADOR', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['28/08/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'JUIZADOS ESPECIAIS CÍVEIS E CRIMINAIS', 'JUIZADO ESPECIAL CÍVEL CENTRAL', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['28/08/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'JUIZADOS ESPECIAIS CÍVEIS E CRIMINAIS', 'JUIZADO ESPECIAL CÍVEL E CRIMINAL DA ZONA NORTE', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data pa

['28/08/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'VARAS CÍVEIS', 'ESPECIALIZADAS', '5ª VARA DE FAMÍLIA', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['28/08/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'VARAS CÍVEIS', 'ESPECIALIZADAS', '5ª VARA DE FAZENDA PÚBLICA ', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['28/08/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'VARAS CÍVEIS', 'ESPECIALIZADAS', '6ª VARA DE FAMÍLIA ', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['28/08/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'VARAS CÍVEIS', 'NÃO ESPECIALIZADAS ', '10ª VARA CÍVEL', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['28/08/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA D

['28/08/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'TRIBUNAL DE JUSTIÇA ', 'CÂMARA CRIMINAL', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['28/08/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'TRIBUNAL DE JUSTIÇA ', 'ESMARN - ESCOLA DA MAGISTRATURA', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['28/08/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'TRIBUNAL DE JUSTIÇA ', 'PRESIDÊNCIA', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['28/08/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'TRIBUNAL DE JUSTIÇA ', 'PRIMEIRA CÂMARA CÍVEL', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['28/08/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'TRIBUNAL DE JUSTIÇA ', 'PUBLICAÇÃO DE ACORDÃOS ', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data par

['28/08/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'PUBLICAÇÕES A PEDIDO', 'COMARCA DE MOSSORO', '4ª VARA CÍVEL', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['28/08/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'PUBLICAÇÕES A PEDIDO', 'COMARCA DE MOSSORO', '4ª VARA CRIMINAL', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['28/08/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'PUBLICAÇÕES A PEDIDO', 'COMARCA DE MOSSORO', '4ª VARA FAMÍLIA', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['28/08/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'PUBLICAÇÕES A PEDIDO', 'COMARCA DE MOSSORO', '5ª VARA CÍVEL', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['28/08/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'PUBLICAÇÕES A PEDIDO', 'COMARCA DE MOSSORO', '5ª VARA CRIMINAL', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['28/08

['28/08/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'TRIBUNAL REGIONAL DO TRABALHO (TRT)', 'TRIBUNAL PLENO E GABINETES', 'ACÓRDÃOS', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['28/08/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'TRIBUNAL REGIONAL DO TRABALHO (TRT)', 'TRIBUNAL PLENO E GABINETES', 'ATAS', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['28/08/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'TRIBUNAL REGIONAL DO TRABALHO (TRT)', 'TRIBUNAL PLENO E GABINETES', 'AVISOS', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['28/08/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'TRIBUNAL REGIONAL DO TRABALHO (TRT)', 'TRIBUNAL PLENO E GABINETES', 'DESPACHOS', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['28/08/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'TRIBUNAL REGIONAL DO TRABALHO (TRT)', 'TRIBUNAL PLENO E GABINETES', 'RESOLUÇÕES ADMINISTRA

In [9]:
driver.quit()

In [10]:
df = pd.DataFrame(materias,columns=['data_publicacao', 'diario', 'sessao', 'subsessao1', 'subsessao2', 'subsessao3', 'subsessao4', 'subsessão5', 'nome_materia', 'materia'])
df.head()

,data_publicacao,diario,sessao,subsessao1,subsessao2,subsessao3,subsessao4,subsessão5,nome_materia,materia
0,28/08/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (AS...,DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI),,,DEI - EXTRATO DISPENSA DE LICITAÇÃO (TECPEL).doc,http://diariooficial.rn.gov.br/dei/dorn3/docvi...
1,28/08/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (AS...,DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI),DEI ASSEJUR,,Não Possui,Nenhuma matéria encontrada nessa data para ess...
2,28/08/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",CONSELHO DE DESENVOLVIMENTO DO ESTADO (CDE),,,,Não Possui,Nenhuma matéria encontrada nessa data para ess...
3,28/08/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",CONSULTORIA GERAL DO ESTADO (CGE),,,,Não Possui,Nenhuma matéria encontrada nessa data para ess...
4,28/08/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",CONTROLADORIA GERAL DO ESTADO (CONTROL),,,,RESUMO DO 3º TA (Locação de veículos - Prorrog...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...


In [13]:
df[df['nome_materia'] == 'Não Possui'].diario.count()

296

In [14]:
df.shape

(431, 10)

In [15]:
filename = "diario - {}".format(data_formatada.replace('/','_'))
df.to_csv(filename)

# Pegar texto das matérias

In [16]:
materias_df = df[df['nome_materia'] != 'Não Possui']
materias_df.head()

,data_publicacao,diario,sessao,subsessao1,subsessao2,subsessao3,subsessao4,subsessão5,nome_materia,materia
0,28/08/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (AS...,DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI),,,DEI - EXTRATO DISPENSA DE LICITAÇÃO (TECPEL).doc,http://diariooficial.rn.gov.br/dei/dorn3/docvi...
4,28/08/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",CONTROLADORIA GERAL DO ESTADO (CONTROL),,,,RESUMO DO 3º TA (Locação de veículos - Prorrog...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...
7,28/08/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",GABINETE DO VICE-GOVERNADOR,,,,Termo de Dispensa 05/2019 - GVG,http://diariooficial.rn.gov.br/dei/dorn3/docvi...
9,28/08/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETARIA DA INFRA-ESTRUTURA,,,,EXTRATO DO DÉCIMO QUARTO TERMO ADITIVO AO CONT...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...
10,28/08/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETARIA DA INFRA-ESTRUTURA,,,,EXTRATO DO CONTRATO N° 014/2019 – SIN CONTRATO...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...


In [18]:
print(materias_df.materia.head())

0     http://diariooficial.rn.gov.br/dei/dorn3/docvi...
4     http://diariooficial.rn.gov.br/dei/dorn3/docvi...
7     http://diariooficial.rn.gov.br/dei/dorn3/docvi...
9     http://diariooficial.rn.gov.br/dei/dorn3/docvi...
10    http://diariooficial.rn.gov.br/dei/dorn3/docvi...
Name: materia, dtype: object


In [53]:
base_url = 'http://diariooficial.rn.gov.br/dei/dorn3/'
# Specify url
url = 'http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20190928&id_doc=660620'

# Package the request, send the request and catch the response: r
r = requests.get(url)

# Extracts the response as html: html_doc
html_doc = r.text

# create a BeautifulSoup object from the HTML: soup
soup = BeautifulSoup(html_doc, "lxml")
frame = soup.find_all('frame')[1]
frame_link = frame.get('src')
frame_link

'documentos/00000001/20190928/660620.htm'

In [60]:
r = requests.get((base_url+frame_link))
html_doc = r.text
noticia = BeautifulSoup(html_doc, "lxml").text.strip()
conteudo_materia = re.sub("(<!--.*?-->)", "", noticia, flags=re.DOTALL)
print(conteudo_materia.strip())

Processo nº 00110009.003568/2019-19
Interessado: COORDENADORIA DE PATRIMÔNIO
 
 Termo de Justificativa de Quebra de Ordem
Cronológica de Pagamento
Considerando o art. 5° da Lei 8.666/93 que prevê que o pagamento das
obrigações relativas ao fornecimento de bens, locações, realização de obras e
prestação de serviços, da Administração Pública devem obedecer, para cada fonte
diferenciada de recursos, a estrita ordem cronológica das datas de suas
exigibilidades, salvo quando presentes relevantes razões de interesse público e
mediante prévia justificativa da autoridade competente, devidamente publicada.
Considerando o Art. 15, III e IV da Resolução do TCE n° 32/2016, que
prevê a admissibilidade da quebra da ordem cronológica de pagamentos tão
somente em caso de calamidade pública ou mediante relevante interesse público,
mediante deliberação expressa e fundamentada do ordenador de despesas.
Considerando a regulamentação da obrigatoriedade de observância da ordem
cronológica de pagamentos, por

# O que fazer
### Seleção de data
    Iterar por data (opcional)
### Criar Funções (fazendo)
### Tratar os links para pegar texto (fazendo)
### Minimizar gambiarras
### Criar arquivo .py

# Teste

Pegar subsessões

In [103]:
elements = driver.find_element_by_id("Menu1_dgMenu_ctl02_LinkButton1")
elements.click()
tabela = driver.find_element_by_id("dgDocumentos")
subsessoes = tabela.find_elements_by_tag_name("a")
for s in subsessoes:
    print(s.text)

CONTRATOS, EDITAIS E AVISOS
GOVERNADORIA
SECRETARIAS DE ESTADO


Pegar links html

In [54]:
elements = driver.find_element_by_id("Menu1_dgMenu_ctl04_LinkButton1")
print(elements)
elements.click()
tabela = driver.find_element_by_id("dgDocumentos")
files = tabela.find_elements_by_tag_name("a")
links = [f.get_attribute("href") for f in files]
links

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="baa570e3-c43b-47b4-9cb5-0e844586e406", element="00bc14c9-d812-4d3b-9a60-da3689727de3")>


['http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20190926&id_doc=660291',
 'http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20190926&id_doc=660292',
 'http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20190926&id_doc=660293']

Caminhar até o fim

In [52]:
elements = driver.find_element_by_id('Menu1_dgMenu')
sessoes = elements.find_elements_by_tag_name('a')

subsessoes_1 = navegar(sessoes[0])
subsessoes_2 = navegar(subsessoes_1[0])
subsessoes_3 = navegar(subsessoes_2[0])
subsessoes_4 = navegar(subsessoes_3[0])
subsessoes_5 = navegar(subsessoes_4[0])
print(subsessoes_5)
driver.back()
driver.back()
driver.back()
#subsessoes_6 = navegar(subsessoes_1[1])

[<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="baa570e3-c43b-47b4-9cb5-0e844586e406", element="0bb13e6b-92a1-46b8-9527-b68ef5f5c7e1")>, <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="baa570e3-c43b-47b4-9cb5-0e844586e406", element="621f55c5-3458-47c2-a793-12f7396cfc6d")>, <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="baa570e3-c43b-47b4-9cb5-0e844586e406", element="32f21de6-eeba-4a32-bc08-a1a015bb5532")>, <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="baa570e3-c43b-47b4-9cb5-0e844586e406", element="0b66c6b2-57f6-4693-a21a-8d03270e5555")>, <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="baa570e3-c43b-47b4-9cb5-0e844586e406", element="80e6c1a8-ea54-4ed6-9e3f-3dc807c79d0c")>, <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="baa570e3-c43b-47b4-9cb5-0e844586e406", element="68ea011a-fd01-4355-9e9a-3200c82115c8")>, <selenium.webdriver.firefox.webelement.FirefoxWebElement (sessi

In [53]:
elements = driver.find_element_by_id('Menu1_dgMenu')
elements = driver.find_element_by_id("Menu1_dgMenu_ctl04_LinkButton1")
elements

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="baa570e3-c43b-47b4-9cb5-0e844586e406", element="927f94cd-4a14-46bd-b368-9c3c04fe8f0d")>

In [70]:
elementos = ["Menu1_dgMenu_ctl0" + str(i) + "_LinkButton1" for i in range(2,10)]
sessoes = driver.find_elements_by_id(elementos[0])
print(sessoes)
subsessoes_1 = navegar(sessoes[0])
elementos2 = ["dgSubSessoes_ctl0"+str(i)+"_Linkbutton1" for i in range(2,len(subsessoes_1)+2)]
print(elementos2)
sessoes2 = driver.find_elements_by_id(elementos2[0])
print(sessoes2)
subsessoes_2 = navegar(sessoes2[0])
elementos3 = ["dgSubSessoes_ctl0"+str(i)+"_Linkbutton1" for i in range(2,len(subsessoes_2)+2)]
print(elementos3)
sessoes3 = driver.find_elements_by_id(elementos3[0])
print(sessoes3)
subsessoes_3 = navegar(sessoes3[0])
subsessoes_4 = navegar(subsessoes_3[0])
subsessoes_5 = navegar(subsessoes_4[0])
print(subsessoes_5)
driver.back()
driver.back()
driver.back()
sessoes6 = driver.find_elements_by_id(elementos3[1])
subsessoes_6 = navegar(sessoes6[0])
print(subsessoes_6)
#baa570e3-c43b-47b4-9cb5-0e844586e406
#dd0912d6-54a2-42fd-a565-5212ce6bdb4d

[<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="baa570e3-c43b-47b4-9cb5-0e844586e406", element="54215378-e6da-4851-b111-4eb108ad7947")>]
['dgSubSessoes_ctl02_Linkbutton1', 'dgSubSessoes_ctl03_Linkbutton1', 'dgSubSessoes_ctl04_Linkbutton1']
[<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="baa570e3-c43b-47b4-9cb5-0e844586e406", element="d08cdc93-a296-4326-9f1a-c8829a1e3736")>]
['dgSubSessoes_ctl02_Linkbutton1', 'dgSubSessoes_ctl03_Linkbutton1', 'dgSubSessoes_ctl04_Linkbutton1', 'dgSubSessoes_ctl05_Linkbutton1', 'dgSubSessoes_ctl06_Linkbutton1', 'dgSubSessoes_ctl07_Linkbutton1', 'dgSubSessoes_ctl08_Linkbutton1', 'dgSubSessoes_ctl09_Linkbutton1', 'dgSubSessoes_ctl010_Linkbutton1', 'dgSubSessoes_ctl011_Linkbutton1', 'dgSubSessoes_ctl012_Linkbutton1', 'dgSubSessoes_ctl013_Linkbutton1', 'dgSubSessoes_ctl014_Linkbutton1', 'dgSubSessoes_ctl015_Linkbutton1', 'dgSubSessoes_ctl016_Linkbutton1', 'dgSubSessoes_ctl017_Linkbutton1', 'dgSubSessoes_ctl018_Linkbutto